# **Necessary Libraries**

In [1]:
import pandas as pd
import os
from keras import optimizers
from keras import layers,models
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, Flatten
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
print(os.listdir("../input"))
import numpy as np

Using TensorFlow backend.


['train', 'test', 'train.csv', 'sample_submission.csv']


# Load the datas

In [2]:
train_dataframe = pd.read_csv("../input/train.csv")
train_dataframe["has_cactus"] = np.where(train_dataframe["has_cactus"] == 1, "yes", "no")
print(train_dataframe.head())

                                     id has_cactus
0  0004be2cfeaba1c0361d39e2b000257b.jpg        yes
1  000c8a36845c0208e833c79c1bffedd1.jpg        yes
2  000d1e9a533f62e55c289303b072733d.jpg        yes
3  0011485b40695e9138e92d0b3fb55128.jpg        yes
4  0014d7a11e90b62848904c1418fc8cf2.jpg        yes


In [3]:
base_dir = "../input/"
train_dir = os.path.join(base_dir,"train/train")
test_dir = os.path.join(base_dir, "test/test")
df_test=pd.read_csv('../input/sample_submission.csv')

testing_dir = os.path.join(base_dir, "test")
batch_size=150

# Splitting the data into Train,Test,Validation

In [4]:
train_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.10,rotation_range=40,
    width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,
    zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')

In [5]:
train_generator = train_datagen.flow_from_dataframe(dataframe = train_dataframe,directory = train_dir,x_col="id",
    y_col="has_cactus",target_size=(150,150),subset="training",batch_size=250,shuffle=True,class_mode="binary"
)

valid_generator = train_datagen.flow_from_dataframe(dataframe = train_dataframe,directory = train_dir,x_col="id",
    y_col="has_cactus",target_size=(150,150),subset="validation",batch_size=125,shuffle=True,class_mode="binary"
)

Found 15750 images belonging to 2 classes.
Found 1750 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(
    rescale=1/255
)

In [7]:
test_generator = test_datagen.flow_from_directory(testing_dir,target_size=(150,150),batch_size=1,
    shuffle=False,class_mode=None)

Found 4000 images belonging to 1 classes.


## Creating the Model

In [8]:
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(Dropout(0.2))
model.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)    

# Compiling the model with loss and optimizers

In [9]:
model.compile(loss='binary_crossentropy',optimizer=optimizers.rmsprop(),metrics=['acc'])

# Fitting the model with train and validation data 

In [10]:
epochs=10
history=model.fit_generator(train_generator,steps_per_epoch=100,epochs=10,validation_data=valid_generator,validation_steps=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
100/100 [==============================] - 194s 2s/step - loss: 1.1899 - acc: 0.8014 - val_loss: 0.7745 - val_acc: 0.5802
Epoch 2/10
100/100 [==============================] - 158s 2s/step - loss: 0.2347 - acc: 0.9109 - val_loss: 0.1884 - val_acc: 0.9251
Epoch 3/10
100/100 [==============================] - 156s 2s/step - loss: 0.1918 - acc: 0.9203 - val_loss: 0.1461 - val_acc: 0.9394
Epoch 4/10
100/100 [==============================] - 157s 2s/step - loss: 0.1773 - acc: 0.9317 - val_loss: 0.1422 - val_acc: 0.9451
Epoch 5/10
100/100 [==============================] - 156s 2s/step - loss: 0.1550 - acc: 0.9384 - val_loss: 0.1110 - val_acc: 0.9547
Epoch 6/10
100/100 [==============================] - 158s 2s/step - loss: 0.1433 - acc: 0.9429 - val_loss: 0.1478 - val_acc: 0.9486
Epoch 7/10
100/100 [==============================] - 155s 2s/step - loss: 0.1464 - acc: 0.9432 - val_loss: 0.1237 - val_acc: 0.9566
Epoch 8/10
100/100 [=

# Predicting the trained model with test data

In [11]:
preds = model.predict_generator(
    test_generator,
    steps=len(test_generator.filenames))

# Making submission in csv format with the trained model of test dataset

In [12]:
df=pd.DataFrame({'id':df_test['id'] })
df['has_cactus']=preds
df.to_csv("submission.csv",index=False)

In [14]:
df.head()

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,0.565943
1,0017242f54ececa4512b4d7937d1e21e.jpg,0.999998
2,001ee6d8564003107853118ab87df407.jpg,0.552546
3,002e175c3c1e060769475f52182583d0.jpg,0.646088
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.564927
